In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Relations

Demonstrates how to create relationships between different portfolios.

Attributes
----------
relations
"""

toggle_code("Hide docstring")

In [2]:
import lusid as lu
import lusid.models as lm
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import json
import os
import pandas as pd

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.8130.0


In [3]:
scope = "relations-example"

# Portfolios

Set up the following portfolios with the following relations

![title](relations.png)

* `master` fund with a relation to 3 sub-funds
* `subaccount1` related to `subaccount1-monthly-postions` representing a version containing monthly positions 
* `subaccount1` related to `subaccount1-abor` representing an ABOR view

In [4]:
portfolios_df = pd.DataFrame(
    data=[
        ["master", "master"],
        ["subaccount1", "subaccount1"],
        ["subaccount2", "subaccount2"],
        ["subaccount3", "subaccount3"],
        ["subaccount1-monthly-postions", "subaccount1-monthly-postions"],
        ["subaccount1-abor", "subaccount1-abor"]
    ], columns=["portfolio_code", "portfolio_name"]
)

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=portfolios_df,
    mapping_required={
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    mapping_optional={
        "created": "$2020-01-01T00:00:00+00:00"
    },
    file_type="portfolios",
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])

,success,failed
0,6,0


## Relations

In [5]:
relation_definitions_api = api_factory.build(lu.RelationDefinitionsApi)
relations_api = api_factory.build(lu.RelationsApi)

## Relationship definition

In [6]:
def create_relation_definition(code, display_name, outward_description, inward_description):
    try:
        relation_definitions_api.create_relation_definition(
            create_relation_definition_request=lu.CreateRelationDefinitionRequest(
                scope=scope, 
                code=code,
                source_entity_domain="Portfolio", 
                target_entity_domain="Portfolio", 
                display_name=display_name, 
                outward_description=outward_description, 
                inward_description=inward_description, 
                life_time="TimeVariant"
            )
        )
        print(f"created relation {scope}/{code}")
    except lu.ApiException as e:
        body = json.loads(e.body)
        if body["code"] != 667:  # RelationDefinitionAlreadyExists
            print(body)
        else:
            print(f"relation {scope}/{code} already exists")

Create the Relation definitions, these are used when creating a relationship between entities

In [7]:
create_relation_definition("subfund", "Master fund link to sub-fund", "parent of", "sub-fund of")

relation relations-example/subfund already exists


In [8]:
create_relation_definition("monthly-positions", "Link to fund containing monthly positions", "daily transactions of", "has monthly positions of")

relation relations-example/monthly-positions already exists


In [9]:
create_relation_definition("abor", "Link to fund containing ABOR", "IBOR of", "ABOR of")

relation relations-example/abor already exists


In [10]:
def create_portfolio_relation(relation, from_portfolio, to_portfolio):
    relations_api.create_relation(
        
        # the scope/code of the RelationDefinition to be created
        scope=scope,
        code=relation,
        
        create_relation_request=lm.CreateRelationRequest(
            source_entity_id={
                # the fields the uniquely identify the source entity
                "Scope": scope,
                "Code": from_portfolio
            }, 
            target_entity_id={
                # the fields the uniquely identify the target entity
                "Scope": scope,
                "Code": to_portfolio
            })
    )

Create the Relation between the portfolios

In [11]:
create_portfolio_relation("subfund", "master", "subaccount1")

In [12]:
create_portfolio_relation("subfund", "master", "subaccount2")

In [13]:
create_portfolio_relation("subfund", "master", "subaccount3")

In [14]:
create_portfolio_relation("monthly-positions", "subaccount1", "subaccount1-monthly-postions")

In [15]:
create_portfolio_relation("abor", "subaccount1", "subaccount1-abor")

## Navigate the master fund relationships

Get the list of portfolio Relations from the `master` porfolio 

In [16]:
portfolios_api = api_factory.build(lu.PortfoliosApi)

In [17]:
sub_funds = portfolios_api.get_portfolio_relations(scope, "master")
lusid_response_to_data_frame(sub_funds.values)

,relation_definition_id.scope,relation_definition_id.code,related_entity_id.EntityType,related_entity_id.Scope,related_entity_id.Code,traversal_direction,traversal_description,effective_from
0,relations-example,subfund,Portfolio,relations-example,subaccount2,Out,parent of,0001-01-01 00:00:00+00:00
1,relations-example,subfund,Portfolio,relations-example,subaccount3,Out,parent of,0001-01-01 00:00:00+00:00
2,relations-example,subfund,Portfolio,relations-example,subaccount1,Out,parent of,0001-01-01 00:00:00+00:00


## `subaccount1` relations

Get the relations for `subaccount1`. There is the Relation `in` from portfolio `master`, and 2 `out` Relations to the `subaccount1-monthly-postions` and `subaccount1-abor` portfolios

In [18]:
subaccount1_relations = portfolios_api.get_portfolio_relations(scope, "subaccount1")
display(lusid_response_to_data_frame(subaccount1_relations.values))

,relation_definition_id.scope,relation_definition_id.code,related_entity_id.EntityType,related_entity_id.Scope,related_entity_id.Code,traversal_direction,traversal_description,effective_from
0,relations-example,monthly-positions,Portfolio,relations-example,subaccount1-monthly-postions,Out,daily transactions of,0001-01-01 00:00:00+00:00
1,relations-example,abor,Portfolio,relations-example,subaccount1-abor,Out,IBOR of,0001-01-01 00:00:00+00:00
2,relations-example,subfund,Portfolio,relations-example,master,In,sub-fund of,0001-01-01 00:00:00+00:00


## Get the related monthly positions for `subaccount1`

In [19]:
monthly_relation = list(filter(lambda relation: relation.related_entity_id["Code"] == "subaccount1-monthly-postions", subaccount1_relations.values))

portfolio = portfolios_api.get_portfolio(
    scope=monthly_relation[0].related_entity_id["Scope"],
    code=monthly_relation[0].related_entity_id["Code"],
)

pd.DataFrame([{
    "scope": portfolio.id.scope,
    "code": portfolio.id.code,
    "display_name": portfolio.display_name
}])

,scope,code,display_name
0,relations-example,subaccount1-monthly-postions,subaccount1-monthly-postions


## Get the group membership for `subaccount1`

In [20]:
def relation_filter(relation):
    return relation.related_entity_id["Code"] == "master" and relation.traversal_direction == "In"

group_relation = list(filter(relation_filter, subaccount1_relations.values))

portfolio = portfolios_api.get_portfolio(
    scope=group_relation[0].related_entity_id["Scope"],
    code=group_relation[0].related_entity_id["Code"],
)

pd.DataFrame([{
    "scope": portfolio.id.scope,
    "code": portfolio.id.code,
    "display_name": portfolio.display_name
}])

,scope,code,display_name
0,relations-example,master,master


## Tear down

In [21]:
for index, row in portfolios_df.iterrows():
        
    portfolios_api.delete_portfolio(scope=scope, code=row.portfolio_code)